# Kaggle
- Kaggle 연결해서 Colab으로 데이터 다운로드하기
- [구글 Colab에 Kaggle 데이터셋 가져오는 방법](https://gannigoing.medium.com/%EA%B5%AC%EA%B8%80-colab%EC%97%90-kaggle-%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%85%8B-%EA%B0%80%EC%A0%B8%EC%98%A4%EB%8A%94-%EB%B0%A9%EB%B2%95-e962b4d687a5)

1. Install

In [ ]:
!pip install kaggle --upgrade

2. Kaggle - 본인 Account 페이지 - Create New API Token 클릭 후, 저장된 kaggle.json 파일을 코랩에 업로드

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"kyuleejeon","key":"21508b111885d8b7b5a6397ace7abe96"}'}

3. kaggle.json 파일의 위치 변경

In [22]:
import os
os.getcwd()

'/content/drive/MyDrive/Colab Notebooks/DICOM_characterization_pilot'

In [23]:
cd /content/drive/MyDrive/Colab Notebooks/DICOM_characterization_pilot/

/content/drive/MyDrive/Colab Notebooks/DICOM_characterization_pilot


In [ ]:
!mkdir -p ~/.kaggle # directory 생성

In [ ]:
!cp kaggle.json ~/.kaggle/ # cp file1 file2: file1 파일을 똑같이 복사하여 file2를 생성한다

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

4. 잘 저장되었는지 확인
- kaggle.json 파일명 나오면 여기까지 성공

In [ ]:
!ls -1ha kaggle.json

kaggle.json


5. 필요한 데이터셋 불러오기
- [RSNA-MICCAI Brain Tumor Radiogenomic Classification](https://www.kaggle.com/competitions/rsna-miccai-brain-tumor-radiogenomic-classification/rules)
- 선정한 이유: [JHU Tutorial](https://pm.jh.edu/cookbook/Demo_Access_DICOMs.html) 따라해보기 위함
- 403 Forbidden Error가 날 경우, [Rules]에서 'I understand and accept' 버튼 클릭


In [ ]:
# 231205
#!kaggle competitions download -c rsna-miccai-brain-tumor-radiogenomic-classification

100% 12.3G/12.3G [10:09<00:00, 14.7MB/s]
100% 12.3G/12.3G [10:09<00:00, 21.7MB/s]


In [ ]:
# 231211 : https://www.kaggle.com/datasets/raddar/nodules-in-chest-xrays-lidcidri
!kaggle datasets download -d raddar/nodules-in-chest-xrays-lidcidri

100% 2.49G/2.50G [00:38<00:00, 50.3MB/s]
100% 2.50G/2.50G [00:38<00:00, 70.3MB/s]


6. 잘 저장되었는지 확인 후 필요하다면 unzip

In [ ]:
!ls

 kaggle.json				 sample_submission.csv
'Kaggle-Jupyter Tutorial_231205.ipynb'	 XNAT-JupyterTutorial_231115.ipynb
 nodules-in-chest-xrays-lidcidri.zip


In [ ]:
#!unzip rsna-miccai-brain-tumor-radiogenomic-classification.zip # 적당히 하다가 스토리지 문제로 끊음
!unzip nodules-in-chest-xrays-lidcidri.zip # 2.5GB

# DICOM C-STORE (23.12.11)
- Any DICOM C-STORE SCU, including scanner consoles or DICOM applications like OsiriX or DicomBrowser, can send files directly to this server.
- DICOM C-STORE receiver (SCP) Specifications
  - ID:
  - AE Title: XNAT
  - Port: 8104
- 시도했으나, unable to connect to remote로 실패함

In [30]:
# fname to list
image_folder = '/content/drive/MyDrive/Colab Notebooks/DICOM_characterization_pilot/images/images/'
image_data = os.listdir(image_folder)
image_data.sort()

print(image_data[0], image_data[-1])
print(len(image_data)) # 463

LIDC-IDRI-0001-000001.dcm LIDC-IDRI-0291-000001.dcm
463


In [17]:
!pip install -U pynetdicom

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 41.8 MB/s eta 0:00:00


In [18]:
from pydicom import dcmread

from pynetdicom import AE, debug_logger # associate with a peer DICOM Application Entity
from pynetdicom.sop_class import CTImageStorage # request the transfer of a single CT datasets

In [19]:
debug_logger()

In [35]:
# Initialise the Application Entity
ae = AE('XNAT')

In [36]:
# Add a requested presentation context
ae.add_requested_context(CTImageStorage)

In [32]:
from tqdm import tqdm
import time

In [37]:
# Read in our DICOM CT datasets
for i in tqdm(range(len(image_data))):
  ds = dcmread(image_folder+image_data[i])
  # Associate with AE at IP 192.168.152.83 and port 8104
  assoc = ae.associate("192.168.152.83", 8104)
  if assoc.is_established:
      # Use the C-STORE service to send the dataset
      # returns the response status as a pydicom Dataset
      status = assoc.send_c_store(ds)

      # Check the status of the storage request
      if status:
          # If the storage request succeeded this will be 0x0000
          print('C-STORE request status: 0x{0:04x}'.format(status.Status))
      else:
          print('Connection timed out, was aborted or received invalid response')

      # Release the association
      assoc.release()
  else:
      print('Association rejected, aborted or never connected')

  0%|          | 0/463 [00:00<?, ?it/s]I: Requesting Association
INFO:pynetdicom.assoc:Requesting Association
E: Association request failed: unable to connect to remote
ERROR:pynetdicom.transport:Association request failed: unable to connect to remote
E: TCP Initialisation Error: [Errno 110] Connection timed out
ERROR:pynetdicom.transport:TCP Initialisation Error: [Errno 110] Connection timed out
I: Aborting Association
INFO:pynetdicom.assoc:Aborting Association
  0%|          | 1/463 [02:11<16:51:08, 131.32s/it]I: Requesting Association
INFO:pynetdicom.assoc:Requesting Association


Association rejected, aborted or never connected


  0%|          | 1/463 [04:18<33:08:24, 258.24s/it]


KeyboardInterrupt: ignored